In [ ]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.2/431.2 kB 5.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 27.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 kB 6.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 31.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 2.9 MB/s eta 0:00:00a 0:00:01


In [6]:
!unzip data.zip

Archive:  data.zip
   creating: data/dev/
  inflating: data/dev/EP3284738A1.txt  
  inflating: data/dev/EP3305797A1.txt  
  inflating: data/dev/EP3312182A1.txt  
  inflating: data/dev/EP3323817A1.txt  
  inflating: data/dev/EP3330263A1.txt  
  inflating: data/dev/EP3674305A1.txt  
  inflating: data/dev/EP3708563A1.txt  
  inflating: data/dev/US10654800B1.txt  
  inflating: data/dev/US10815179B1.txt  
  inflating: data/dev/US20180002276A1.txt  
  inflating: data/dev/US20180008574A1.txt  
  inflating: data/dev/US20180016269A1.txt  
  inflating: data/dev/US20180057448A1.txt  
  inflating: data/dev/US20180072767A1.txt  
  inflating: data/dev/US20180083209A1.txt  
  inflating: data/dev/US20180092937A1.txt  
  inflating: data/dev/US20180105741A1.txt  
  inflating: data/dev/US20200031833A1.txt  
  inflating: data/dev/US20200048291A1.txt  
  inflating: data/dev/US20200055807A1.txt  
  inflating: data/dev/US20200121691A1.txt  
  inflating: data/dev/US20200123187A1.txt  
  inflating: data/dev/US

In [23]:
import os
import os.path as osp
import nltk
import random
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
english_stopwords = stopwords.words("english")
import numpy as np
import re
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt
import pandas as pd
import string
import torch
import datasets
from datasets import load_dataset
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import multiprocessing
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dev_path = "data/dev"
train_path = "data/train"

dev_label_path = "dev_labels.pkl"
train_label_path = "train_labels.pkl"

In [3]:
dev_files = [osp.join(dev_path, f)  for f in os.listdir(dev_path) if f[-3:]=='txt']
print(len(dev_files))
print(dev_files)

30
['data/dev/US20180072767A1.txt', 'data/dev/US20200121691A1.txt', 'data/dev/EP3323817A1.txt', 'data/dev/EP3284738A1.txt', 'data/dev/US20180002276A1.txt', 'data/dev/US20200048291A1.txt', 'data/dev/US20200270285A1.txt', 'data/dev/US20200165244A1.txt', 'data/dev/US20200123187A1.txt', 'data/dev/US20200308177A1.txt', 'data/dev/US20200148885A1.txt', 'data/dev/US20200031833A1.txt', 'data/dev/US10654800B1.txt', 'data/dev/US20180092937A1.txt', 'data/dev/EP3674305A1.txt', 'data/dev/US20180083209A1.txt', 'data/dev/US10815179B1.txt', 'data/dev/EP3312182A1.txt', 'data/dev/EP3330263A1.txt', 'data/dev/US20200055807A1.txt', 'data/dev/EP3708563A1.txt', 'data/dev/US20180105741A1.txt', 'data/dev/US20200384451A1.txt', 'data/dev/US20200181061A1.txt', 'data/dev/US20200164075A1.txt', 'data/dev/US20200377536A1.txt', 'data/dev/US20180008574A1.txt', 'data/dev/US20180016269A1.txt', 'data/dev/EP3305797A1.txt', 'data/dev/US20180057448A1.txt']


We keep the dev set for Testing and take 20 documents out of the 120 documents in the train set for validation

In [4]:
train_files = [osp.join(train_path, f) for f in os.listdir(train_path) if f[-3:]=='txt']
print(len(train_files))
random.shuffle(train_files)

val_files = train_files[:20]
train_files = train_files[20:]

print(len(train_files))
print(len(val_files))

120
100
20


In [63]:
def create_dataset(files, labels_dict, output_file, test_file=False):

  if(test_file):
      train_data = {'para':[], 'label':[], 'document':[]}
  else:
      train_data = {'para':[], 'label':[]}

  for f in files:
    with open(f, encoding="utf-8" ) as fp:
        
      lines = fp.readlines()
      train_data['para'].extend(lines)

      labels = labels_dict[f.split('/')[-1][:-4]] #".txt"
      train_data['label'].extend(labels)
        
      if(test_file):
          train_data['document'].extend([f for i in range(len(lines))])
      

  nl = []
  for l in train_data['label']:
    if(l==1):
      nl.append("1")
    else:
      nl.append("0")
    
  train_data['label'] = nl

  df = pd.DataFrame(train_data)
  print(df.head())
  df.to_csv(output_file, index = False)


In [6]:
# create_dataset(dev_files, pickle.load(open(dev_label_path, "rb")), "dev_data.csv") # test set
create_dataset(dev_files, pickle.load(open(dev_label_path, "rb")), "dev_data_annot.csv", test_file=True) # test set
create_dataset(train_files, pickle.load(open(train_label_path, "rb")), "train_data.csv") #train_set
create_dataset(val_files, pickle.load(open(train_label_path, "rb")), "val_data.csv") # validation set

                                                para label
0                                  TECHNICAL FIELD\n     0
1  The present disclosure relates to trehalose an...     0
2                                       BACKGROUND\n     0
3  Trehalose (D-glucose-α-1,1-α-D-glucose) is a d...     0
4                                          SUMMARY\n     0
                                                para label
0           CROSS-REFERENCE TO RELATED APPLICATION\n     0
1  This application claims priority benefits to C...     0
2                           FIELD OF THE INVENTION\n     0
3  The present invention relates to the technical...     0
4                      BACKGROUND OF THE INVENTION\n     0
                                                para label
0                                  TECHNICAL FIELD\n     0
1  The present invention relates to a novel phtha...     0
2                                   BACKGROUND ART\n     0
3  In recent years, a near-infrared absorbing mat...    

## Base BERT

In [7]:
#load csv files
dataset = load_dataset("csv", 
                       data_files={"train": "train_data.csv",
                                   "test": "val_data.csv"})
dataset

Using custom data configuration default-3bffab270db04a4f


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/csv/default-3bffab270db04a4f/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['para', 'label'],
        num_rows: 44195
    })
    test: Dataset({
        features: ['para', 'label'],
        num_rows: 9368
    })
})

In [8]:
# model we will be using throughout
pretrained_model = "bert-base-uncased"

#load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, do_lower=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
s = "2-ethynylpyridine was replaced by 2-ethynyl-6-methylpyridine, 4-fluoroaniline was replaced by 3-aminopyridine, while the remaining raw materials, reagents and the preparation method were the same as those in Example 1 to give the product ZD080, yield 80%."
tokenized_output = tokenizer(s)
print("tokenizer output:\n", tokenized_output, "\n")
print("input_ids to tokens:\n", tokenizer.convert_ids_to_tokens(tokenized_output["input_ids"]))

tokenizer output:
 {'input_ids': [101, 1016, 1011, 3802, 10536, 4890, 14277, 12541, 28173, 2638, 2001, 2999, 2011, 1016, 1011, 3802, 10536, 4890, 2140, 1011, 1020, 1011, 25003, 7685, 14615, 3170, 1010, 1018, 1011, 19857, 14604, 7088, 4179, 2001, 2999, 2011, 1017, 1011, 13096, 7685, 14615, 3170, 1010, 2096, 1996, 3588, 6315, 4475, 1010, 2128, 4270, 7666, 1998, 1996, 7547, 4118, 2020, 1996, 2168, 2004, 2216, 1999, 2742, 1015, 2000, 2507, 1996, 4031, 1062, 2094, 2692, 17914, 1010, 10750, 3770, 1003, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [10]:
def preprocess(examples):
    return tokenizer(examples["para"], truncation=True, 
                     padding="max_length", max_length=128,
                     return_token_type_ids=False)

n_cpu = multiprocessing.cpu_count()

#preprocess
dataset = dataset.map(preprocess,# preprocessing function
                      num_proc=n_cpu,# number of workers to use
                      remove_columns=["para"] # columns to remove after preprocessing
                     )
dataset

#0:   0%|          | 0/11049 [00:00<?, ?ex/s]

#1:   0%|          | 0/11049 [00:00<?, ?ex/s]

#2:   0%|          | 0/11049 [00:00<?, ?ex/s]

#3:   0%|          | 0/11048 [00:00<?, ?ex/s]

#0:   0%|          | 0/2342 [00:00<?, ?ex/s]

#1:   0%|          | 0/2342 [00:00<?, ?ex/s]

#2:   0%|          | 0/2342 [00:00<?, ?ex/s]

#3:   0%|          | 0/2342 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 44195
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 9368
    })
})

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(x):
    predictions, targets = x
    predictions = predictions.argmax(axis=1)
    scores = precision_recall_fscore_support(
        targets, predictions, average="weighted", zero_division=0.
        )
    
    return {
        "accuracy": (predictions == targets).sum() / len(predictions),
        "precision": scores[0],
        "recall": scores[1],
        "f-1": scores[2]
    }

batch_size = 64 # defined previously
seed = 8 # defined previously
lr = 5e-5 # learning rate
weight_decay = 1e-5
warmup_steps = 10 # scheduler #warmup steps
n_epochs = 5  # for the time sake. you would often want to train for longer epochs.

args = TrainingArguments(
    output_dir = "trained_model",
    evaluation_strategy="epoch",
    save_strategy="epoch", # could be epoch or step
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=n_epochs,
    weight_decay=weight_decay,
    warmup_steps=warmup_steps,
    load_best_model_at_end=True,
    metric_for_best_model="f-1", # metric defined in compute_metrics function
    seed=seed,
    dataloader_num_workers=n_cpu,
    disable_tqdm=False,
    save_total_limit=5, #limit number of checkpoints saved. Otherwise could take a lot of storage
)

trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# start training
# metric shown below are for validation set
trainer.train()

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 44195
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3455
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method t

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F-1
1,0.182300,0.330488,0.915137,0.923918,0.915137,0.917122
2,0.111400,0.315556,0.922289,0.929247,0.922289,0.923907
3,0.071200,0.282117,0.921008,0.922697,0.921008,0.921626
4,0.059200,0.360187,0.920474,0.924071,0.920474,0.921552
5,0.058100,0.371659,0.925491,0.928143,0.925491,0.926329


***** Running Evaluation *****
  Num examples = 9368
  Batch size = 64
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint t

TrainOutput(global_step=3455, training_loss=0.08863994637032493, metrics={'train_runtime': 4518.4456, 'train_samples_per_second': 48.905, 'train_steps_per_second': 0.765, 'total_flos': 1.4535241364544e+16, 'train_loss': 0.08863994637032493, 'epoch': 5.0})

In [13]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 9368
  Batch size = 64
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.3716585636138916,
 'eval_accuracy': 0.9254910333048676,
 'eval_precision': 0.928143184465625,
 'eval_recall': 0.9254910333048676,
 'eval_f-1': 0.926328918695548,
 'eval_runtime': 63.8799,
 'eval_samples_per_second': 146.65,
 'eval_steps_per_second': 2.301,
 'epoch': 5.0}

In [14]:
model.cpu().save_pretrained("base-bert-patent-finetuned")
model.to(device)

Configuration saved in base-bert-patent-finetuned/config.json
Model weights saved in base-bert-patent-finetuned/pytorch_model.bin


In [65]:
def evaluate_test(val_file, model):
    dataset_pt = load_dataset("csv", 
                          data_files={"val": val_file})

    dataset_pt = dataset_pt.map(preprocess,#preprocessing function
                                num_proc=2,#number of workers to use
                                remove_columns=["para"] #column to remove upon preprocessing
                                )
    print(dataset_pt)
    dataset_pt.set_format(type='torch', columns=['label', 'input_ids', 'attention_mask'])
    
    batch_size = 64

    test_dataloader = torch.utils.data.DataLoader(
        dataset_pt["val"], batch_size=batch_size, shuffle=False, num_workers=n_cpu, drop_last = False)
    
    model.eval()

    with torch.no_grad():
        all_predictions = []
        all_targets = []
        for i, batch in enumerate(tqdm(test_dataloader, desc="Test Epoch", leave=False)):
            targets = batch["label"]
            batch = {k: v.to(device) for k, v in batch.items() if k != "label"}
            outputs = model(**batch)
            logits = outputs.logits
            all_predictions.extend(logits.argmax(axis=1).tolist())
            all_targets.extend(targets.tolist())
                            

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    scores = precision_recall_fscore_support(all_targets, all_predictions, 
                                             average="weighted", zero_division=0.)
    
    test_scores={
        "eval_accuracy": (all_predictions == all_targets).sum() / len(all_predictions),
        "eval_precision": scores[0],
        "eval_recall": scores[1],
        "eval_f-1": scores[2]
    }
    print(test_scores)
    return all_predictions

In [51]:
predictions = evaluate_test("dev_data.csv", model)

Using custom data configuration default-ab5dbed000cde288
Found cached dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-ab5dbed000cde288/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-ab5dbed000cde288/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-83f4b673491cdd53.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-ab5dbed000cde288/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-5eca74ac844d8367.arrow


DatasetDict({
    val: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 12871
    })
})


{'eval_accuracy': 0.896822313728537, 'eval_precision': 0.8937763748107974, 'eval_recall': 0.896822313728537, 'eval_f-1': 0.8935587571694755}


In [66]:
df = pd.read_csv('dev_data_annot.csv')
df['predictions'] = predictions
error = df[df['label'] != df['predictions']]
print((len(df)- len(error)) / len(df))

0.896822313728537


In [67]:
print(len(error))
error.head(20)
error.to_csv("errors.csv")

1328


## Chemical domain pretrained BERT

In [68]:
#load csv files
dataset = load_dataset("csv", 
                       data_files={"train": "train_data.csv",
                                   "test": "val_data.csv"})
dataset

Using custom data configuration default-3bffab270db04a4f
Found cached dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-3bffab270db04a4f/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['para', 'label'],
        num_rows: 44195
    })
    test: Dataset({
        features: ['para', 'label'],
        num_rows: 9368
    })
})

In [69]:

# model we will be using throughout
pretrained_model = "recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier"

#load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, do_lower=True)


Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/658 [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--recobo--chemical-bert-uncased-pharmaceutical-chemical-classifier/snapshots/7e514c1d56f117617dbdbc37a0b26ca20a84878a/config.json
Model config BertConfig {
  "_name_or_path": "recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}



Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

loading file vocab.txt from cache at /home/ubuntu/.cache/huggingface/hub/models--recobo--chemical-bert-uncased-pharmaceutical-chemical-classifier/snapshots/7e514c1d56f117617dbdbc37a0b26ca20a84878a/vocab.txt
loading file tokenizer.json from cache at /home/ubuntu/.cache/huggingface/hub/models--recobo--chemical-bert-uncased-pharmaceutical-chemical-classifier/snapshots/7e514c1d56f117617dbdbc37a0b26ca20a84878a/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/ubuntu/.cache/huggingface/hub/models--recobo--chemical-bert-uncased-pharmaceutical-chemical-classifier/snapshots/7e514c1d56f117617dbdbc37a0b26ca20a84878a/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--recobo--chemical-bert-uncased-pharmaceutical-chemical-classifier/snapshots/7e514c1d56f117617dbdbc37a0b26ca20a84878a/tokenizer_config.json
loading configuration file config.json from cache at /home/u

In [70]:
s = "2-ethynylpyridine was replaced by 2-ethynyl-6-methylpyridine, 4-fluoroaniline was replaced by 3-aminopyridine, while the remaining raw materials, reagents and the preparation method were the same as those in Example 1 to give the product ZD080, yield 80%."
tokenized_output = tokenizer(s)
print("tokenizer output:\n", tokenized_output, "\n")
print("input_ids to tokens:\n", tokenizer.convert_ids_to_tokens(tokenized_output["input_ids"]))

tokenizer output:
 {'input_ids': [102, 170, 579, 2311, 819, 654, 11024, 5978, 241, 6703, 214, 170, 579, 2311, 819, 654, 579, 370, 579, 6090, 11024, 5978, 422, 286, 579, 6696, 8212, 4564, 1169, 241, 6703, 214, 239, 579, 4020, 11024, 5978, 422, 969, 111, 4234, 6908, 2518, 422, 10969, 137, 111, 4737, 551, 267, 111, 855, 188, 1052, 121, 1143, 158, 147, 3226, 111, 2378, 25174, 8037, 30131, 422, 2210, 2833, 1863, 205, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 

input_ids to tokens:
 ['[CLS]', '2', '-', 'eth', '##yn', '##yl', '##pyr', '##idine', 'was', 'replaced', 'by', '2', '

In [71]:
def preprocess(examples):
    return tokenizer(examples["para"], truncation=True, 
                     padding="max_length", max_length=128,
                     return_token_type_ids=False)

n_cpu = multiprocessing.cpu_count()

#preprocess
dataset = dataset.map(preprocess,# preprocessing function
                      num_proc=n_cpu,# number of workers to use
                      remove_columns=["para"] # columns to remove after preprocessing
                     )
dataset

#2:   0%|          | 0/11049 [00:00<?, ?ex/s]

#0:   0%|          | 0/11049 [00:00<?, ?ex/s]

#1:   0%|          | 0/11049 [00:00<?, ?ex/s]

#3:   0%|          | 0/11048 [00:00<?, ?ex/s]

#0:   0%|          | 0/2342 [00:00<?, ?ex/s]

#2:   0%|          | 0/2342 [00:00<?, ?ex/s]

#1:   0%|          | 0/2342 [00:00<?, ?ex/s]

#3:   0%|          | 0/2342 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 44195
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 9368
    })
})

In [72]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model)

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--recobo--chemical-bert-uncased-pharmaceutical-chemical-classifier/snapshots/7e514c1d56f117617dbdbc37a0b26ca20a84878a/config.json
Model config BertConfig {
  "_name_or_path": "recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}



Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--recobo--chemical-bert-uncased-pharmaceutical-chemical-classifier/snapshots/7e514c1d56f117617dbdbc37a0b26ca20a84878a/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [73]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(x):
    predictions, targets = x
    predictions = predictions.argmax(axis=1)
    scores = precision_recall_fscore_support(
        targets, predictions, average="weighted", zero_division=0.
        )
    
    return {
        "accuracy": (predictions == targets).sum() / len(predictions),
        "precision": scores[0],
        "recall": scores[1],
        "f-1": scores[2]
    }

batch_size = 64 # defined previously
seed = 8 # defined previously
lr = 5e-5 # learning rate
weight_decay = 1e-5
warmup_steps = 10 # scheduler #warmup steps
n_epochs = 5  # for the time sake. you would often want to train for longer epochs.

args = TrainingArguments(
    output_dir = "trained_model",
    evaluation_strategy="epoch",
    save_strategy="epoch", # could be epoch or step
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=n_epochs,
    weight_decay=weight_decay,
    warmup_steps=warmup_steps,
    load_best_model_at_end=True,
    metric_for_best_model="f-1", # metric defined in compute_metrics function
    seed=seed,
    dataloader_num_workers=n_cpu,
    disable_tqdm=False,
    save_total_limit=5, #limit number of checkpoints saved. Otherwise could take a lot of storage
)

trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# start training
# metric shown below are for validation set
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 44195
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3455
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than usin

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F-1
1,0.178500,0.271304,0.920260,0.923670,0.920260,0.921301
2,0.090100,0.320512,0.917912,0.925515,0.917912,0.919691
3,0.059600,0.309093,0.910120,0.912513,0.910120,0.910978
4,0.051100,0.359465,0.912681,0.915676,0.912681,0.913682
5,0.049200,0.410718,0.910653,0.912073,0.910653,0.911219


***** Running Evaluation *****
  Num examples = 9368
  Batch size = 64
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint t

TrainOutput(global_step=3455, training_loss=0.07796699534966874, metrics={'train_runtime': 4378.0982, 'train_samples_per_second': 50.473, 'train_steps_per_second': 0.789, 'total_flos': 1.4535241364544e+16, 'train_loss': 0.07796699534966874, 'epoch': 5.0})

In [74]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 9368
  Batch size = 64
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.27130433917045593,
 'eval_accuracy': 0.9202604611443211,
 'eval_precision': 0.9236696017528313,
 'eval_recall': 0.9202604611443211,
 'eval_f-1': 0.9213008486981364,
 'eval_runtime': 64.2906,
 'eval_samples_per_second': 145.713,
 'eval_steps_per_second': 2.286,
 'epoch': 5.0}

In [83]:
predictions = evaluate_test("dev_data.csv", model)

Using custom data configuration default-ab5dbed000cde288
Found cached dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-ab5dbed000cde288/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-ab5dbed000cde288/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-8cbd75b5e8195dfc.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-ab5dbed000cde288/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-63baf422b714b6f0.arrow


DatasetDict({
    val: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 12871
    })
})


{'eval_accuracy': 0.9035816952839717, 'eval_precision': 0.9020079733355804, 'eval_recall': 0.9035816952839717, 'eval_f-1': 0.9025884170190543}


In [84]:
df = pd.read_csv('dev_data_annot.csv')
df['predictions'] = predictions
error = df[df['label'] != df['predictions']]
print((len(df)- len(error)) / len(df))

0.9035816952839717


In [85]:
print(len(error))
error.head(20)
error.to_csv("errors_chemical_bert.csv")

1241


In [79]:
model.cpu().save_pretrained("chemical-pharma-bert-patent-finetuned")
model.to(device)
print(5)

Configuration saved in chemical-pharma-bert-patent-finetuned/config.json
Model weights saved in chemical-pharma-bert-patent-finetuned/pytorch_model.bin


5
